Scraping Kenpom to get data on past and current tournament teams
Inspiration code source: https://www.kaggle.com/walterhan/scrape-kenpom-data

# Imports

In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests
import os

# Scraper

In [3]:
# Set Path
PATH = '/Users/matthewdenko/Documents/March_Madness_Predictions/March_Madness/march_madness_2022'
os.chdir(PATH)

# Base url, and a lambda func to return url for a given year
base_url = 'http://kenpom.com/index.php'
url_year = lambda x: '%s?y=%s' % (base_url, str(x) if x != 2022 else base_url)

# Years on kenpom's site (could also scrape this and 
# set as a list if you want to be more dynamic)
years = list(range(2003, 2021))
years.append(2022)

# Create a method that parses a given year and spits out a raw dataframe
def import_raw_year(year):
    """
    Imports raw data from a ken pom year into a dataframe
    """
    f = requests.get(url_year(year))
    soup = BeautifulSoup(f.text)
    table_html = soup.find_all('table', {'id': 'ratings-table'})

    # Weird issue w/ <thead> in the html
    # Prevents us from just using pd.read_html
    # Let's find all the thead contents and just replace/remove them
    # This allows us to easily put the table row data into a dataframe using panda
    thead = table_html[0].find_all('thead')

    table = table_html[0]
    for x in thead:
        table = str(table).replace(str(x), '')

    df = pd.read_html(table)[0]
    df['year'] = year
    return df
    

# Import all the years into a singular dataframe
df = None
for x in years:
    df = pd.concat( (df, import_raw_year(x)), axis=0) \
        if df is not None else import_raw_year(2003)

# Column rename based off of original website
df.columns = ['Rank', 'Team', 'Conference', 'W-L', 'Pyth', 
             'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank',
             'AdjustT', 'AdjustT Rank', 'Luck', 'Luck Rank', 
             'SOS Pyth', 'SOS Pyth Rank', 'SOS OppO', 'SOS OppO Rank',
             'SOS OppD', 'SOS OppD Rank', 'NCSOS Pyth', 'NCSOS Pyth Rank', 'Year']
             
# Lambda that returns true if given string is a number and a valid seed number (1-16)
valid_seed = lambda x: True if str(x).replace(' ', '').isdigit() \
                and int(x) > 0 and int(x) <= 16 else False

# Use lambda to parse out seed/team
df['Seed'] = df['Team'].apply(lambda x: x[-2:].replace(' ', '') \
                              if valid_seed(x[-2:]) else np.nan )


df['Team'] = df['Team'].apply(lambda x: x[:-2] if valid_seed(x[-2:]) else x)

# Split W-L column into wins and losses
df['Wins'] = df['W-L'].apply(lambda x: int(re.sub('-.*', '', x)) )
df['Losses'] = df['W-L'].apply(lambda x: int(re.sub('.*-', '', x)) )
df.drop('W-L', inplace=True, axis=1)


# Reorder columns just cause I'm OCD
df=df[[ 'Year', 'Rank', 'Team', 'Conference', 'Wins', 'Losses', 'Seed','Pyth', 
             'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank',
             'AdjustT', 'AdjustT Rank', 'Luck', 'Luck Rank', 
             'SOS Pyth', 'SOS Pyth Rank', 'SOS OppO', 'SOS OppO Rank',
             'SOS OppD', 'SOS OppD Rank', 'NCSOS Pyth', 'NCSOS Pyth Rank']]
             
df.head(25)

/opt/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/anaconda3/lib/python3.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


,Year,Rank,Team,Conference,Wins,Losses,Seed,Pyth,AdjustO,AdjustO Rank,...,Luck,Luck Rank,SOS Pyth,SOS Pyth Rank,SOS OppO,SOS OppO Rank,SOS OppD,SOS OppD Rank,NCSOS Pyth,NCSOS Pyth Rank
0,2003,1,Kentucky,SEC,32,4,1,29.18,116.5,5,...,0.051,53,11.20,4,108.6,3,97.4,16,6.77,33
1,2003,2,Kansas,B12,30,8,2,28.62,115.0,12,...,-0.017,208,11.84,2,108.6,2,96.8,9,6.07,35
2,2003,3,Pittsburgh,BE,28,5,2,28.61,114.8,14,...,-0.023,224,7.08,56,105.5,66,98.4,48,-8.24,310
3,2003,4,Arizona,P10,28,4,1,26.80,115.6,10,...,-0.007,181,8.69,31,107.2,14,98.5,52,8.19,26
4,2003,5,Illinois,B10,25,7,4,24.47,113.2,22,...,-0.029,242,7.29,52,105.8,60,98.5,53,-4.18,257
5,2003,6,Duke,ACC,26,7,3,23.75,115.0,13,...,-0.030,246,8.85,28,107.1,17,98.3,43,0.64,138
6,2003,7,Texas,B12,26,7,1,23.49,119.3,3,...,-0.009,188,10.36,9,107.1,16,96.7,7,2.16,104
7,2003,8,Syracuse,BE,30,5,3,23.28,114.4,17,...,0.083,16,10.32,10,106.6,34,96.3,4,-3.63,242
8,2003,9,Oklahoma,B12,27,7,1,23.01,112.1,25,...,0.034,84,9.01,24,106.7,30,97.7,25,-2.32,214
9,2003,10,Louisville,CUSA,25,7,4,22.75,115.8,9,...,-0.029,241,6.80,61,105.4,67,98.6,56,6.01,39


# Map Team ID to Kenpom Data

In [4]:
df.Team=df.Team.apply(lambda x: x.replace('-',' '))
df.Team=df.Team.apply(lambda x: x.lower())
df.Team=df.Team.apply(lambda x: x.strip())
df.Team=df.Team.replace('mississippi valley st.','mississippi valley state')
#df.Team=df.Team.replace('texas a&m corpus chris','texas a&m corpus christi')
df.Team=df.Team.replace('dixie st.','dixie st')
df.Team=df.Team.replace('st. francis pa','st francis pa')
df.Team=df.Team.replace('ut rio grande valley','texas rio grande valley')
df.Team=df.Team.replace('southeast missouri st.','southeast missouri state')
df.Team=df.Team.replace('tarleton st.','tarleton st')
df.Team=df.Team.replace('liu','liu brooklyn')
df.Team=df.Team.replace('cal st. bakersfield','cal state bakersfield')

df.Team=df.Team.replace('virginia military inst','virginia military	')
df.Team=df.Team.replace('louisiana saint','louisiana state')
df.Team=df.Team.replace('nj inst of technology','njit')

df.Team=df.Team.replace('texas a&m corpus chris','texas a&m corpus')
df.Team=df.Team.replace('md baltimore county','maryland baltimore county')

df.Team=df.Team.replace('southwest missouri saint','southwest missouri state')
df.Team=df.Team.replace('southwest texas saint','southwest texas saint')
df.Team=df.Team.replace('winston salem saint','winston salem saint')

#-------------------------------------------------------
# merge with spelling file to get the TeamID
spelling=pd.read_csv("inputs/MTeamSpellings.csv",encoding='cp1252')
spelling.columns=['Team','TeamID']
spelling.Team=spelling.Team.apply(lambda x: x.replace('-',' '))
df.Team=df.Team.apply(lambda x: x.strip())


df=df.merge(spelling[['Team','TeamID']],on='Team',how='left')

df.Team=df.Team.apply(lambda x: x.replace('st.','saint'))
df.Team=df.Team.apply(lambda x: x.replace(';',''))
df.Team=df.Team.apply(lambda x: x.replace('\t',''))
df.Team=df.Team.replace('texas a&m corpus chris','texas a&m corpus')
df.Team=df.Team.replace('louisiana saint','louisiana state')
df.Team=df.Team.replace('southwest missouri saint','southwest missouri state')
df.Team=df.Team.replace('southwest texas saint','texas state')
df.Team=df.Team.replace('winston salem saint','winston salem state')

df=df.merge(spelling[['Team','TeamID']],on='Team',how='left')

df.TeamID_x.fillna(df.TeamID_y, inplace=True)


df=df.drop(['TeamID_y','Conference','Wins','Losses','Seed'],axis = 1)
df.head()

,Year,Rank,Team,Pyth,AdjustO,AdjustO Rank,AdjustD,AdjustD Rank,AdjustT,AdjustT Rank,...,Luck Rank,SOS Pyth,SOS Pyth Rank,SOS OppO,SOS OppO Rank,SOS OppD,SOS OppD Rank,NCSOS Pyth,NCSOS Pyth Rank,TeamID_x
0,2003,1,kentucky,29.18,116.5,5,87.4,4,67.7,132,...,53,11.20,4,108.6,3,97.4,16,6.77,33,1246.0
1,2003,2,kansas,28.62,115.0,12,86.4,3,72.2,14,...,208,11.84,2,108.6,2,96.8,9,6.07,35,1242.0
2,2003,3,pittsburgh,28.61,114.8,14,86.2,2,64.7,266,...,224,7.08,56,105.5,66,98.4,48,-8.24,310,1338.0
3,2003,4,arizona,26.80,115.6,10,88.8,8,72.7,11,...,181,8.69,31,107.2,14,98.5,52,8.19,26,1112.0
4,2003,5,illinois,24.47,113.2,22,88.7,7,67.8,127,...,242,7.29,52,105.8,60,98.5,53,-4.18,257,1228.0


In [5]:
df.columns = map(str.lower, df.columns)
df.columns = df.columns.str.replace(' ', '_')
df.head(25)

,year,rank,team,pyth,adjusto,adjusto_rank,adjustd,adjustd_rank,adjustt,adjustt_rank,...,luck_rank,sos_pyth,sos_pyth_rank,sos_oppo,sos_oppo_rank,sos_oppd,sos_oppd_rank,ncsos_pyth,ncsos_pyth_rank,teamid_x
0,2003,1,kentucky,29.18,116.5,5,87.4,4,67.7,132,...,53,11.20,4,108.6,3,97.4,16,6.77,33,1246.0
1,2003,2,kansas,28.62,115.0,12,86.4,3,72.2,14,...,208,11.84,2,108.6,2,96.8,9,6.07,35,1242.0
2,2003,3,pittsburgh,28.61,114.8,14,86.2,2,64.7,266,...,224,7.08,56,105.5,66,98.4,48,-8.24,310,1338.0
3,2003,4,arizona,26.80,115.6,10,88.8,8,72.7,11,...,181,8.69,31,107.2,14,98.5,52,8.19,26,1112.0
4,2003,5,illinois,24.47,113.2,22,88.7,7,67.8,127,...,242,7.29,52,105.8,60,98.5,53,-4.18,257,1228.0
5,2003,6,duke,23.75,115.0,13,91.3,15,70.7,40,...,246,8.85,28,107.1,17,98.3,43,0.64,138,1181.0
6,2003,7,texas,23.49,119.3,3,95.8,58,69.4,67,...,188,10.36,9,107.1,16,96.7,7,2.16,104,1400.0
7,2003,8,syracuse,23.28,114.4,17,91.1,14,71.0,32,...,16,10.32,10,106.6,34,96.3,4,-3.63,242,1393.0
8,2003,9,oklahoma,23.01,112.1,25,89.1,10,62.7,299,...,84,9.01,24,106.7,30,97.7,25,-2.32,214,1328.0
9,2003,10,louisville,22.75,115.8,9,93.0,25,72.5,12,...,241,6.80,61,105.4,67,98.6,56,6.01,39,1257.0


# Export File

In [6]:
df.to_csv('inputs/kenpom.csv',index=False)